In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import metrics
from os import listdir
from os.path import isfile, join
from sklearn.preprocessing import Normalizer
from sklearn.metrics import f1_score

In [2]:
path="C:\\Users\\nilu1\\PatternCracking\\acc\\"
pathex=[]
temp = [f for f in listdir(path) if isfile(join(path, f))]
temp.sort()
for i in range(len(temp)):
    st=temp[i]
    if(st[:2]=="18"):
        pathex.append(temp[i])
# print(pathex)

In [3]:
# n -> number of lines in csv file
# actual -> the actual key pressed
n=25
actual=1
df=pd.read_csv(path+pathex[0])
for i in range(len(pathex)):
    dff=pd.read_csv(path+pathex[i])
    mini=10000
    minkeyd=1000
    minkeyu=1000
    keydi=pathex[i].rfind("_")+4
    keyui=pathex[i].rfind("&")+1
    for j in dff.index:
        t=abs(dff['Time'][j] - int(pathex[i][keydi:keyui-1]))
        if(t < mini):
            mini=t
            minkeyd=j
    mini=10000
    for j in dff.index:
        t=abs(dff['Time'][j] - int(pathex[i][keyui:-4]))
        if(t < mini):
            mini=t
            minkeyu=j
#     print(minkeyd)
#     print(minkeyu)
    window=minkeyu-minkeyd
    left=n-window    
    dff=dff.head(minkeyu + round(left/2))
    dff=dff.tail(n)
    dff.reset_index(inplace=True)
    dff=dff.drop(columns="index")
    dff=dff.drop(columns="Time")
    df1=dff.stack().swaplevel()
    df1.index=df1.index.map('{0[0]}_{0[1]}'.format) 
    dff=df1.to_frame().T
    ind=pathex[i].rfind("_")+3
#     print(pathex[i])
    dff.insert(n*3, "key", int(pathex[i][ind]))
    if(i==0):
        df=dff
    else:
        df=df.append(dff,ignore_index = True)
# df

In [4]:
x = df.iloc[:, 0:n*3].values
y = df.iloc[:, n*3].values

In [5]:
# x = df.drop('Home', axis=1)
# y = df['Home']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)
# sc = Normalizer()
# x_train = sc.fit_transform(x_train)
# x_test = sc.transform(x_test)
logmodel = LogisticRegression(random_state=0, solver="lbfgs", multi_class="multinomial")
logmodel.fit(x_train, y_train)
 
predictions = logmodel.predict(x_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})
df

C:\Users\nilu1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Actual  Predicted
0        9          9
1        6          9
2        7          7
3        6          9
4        1          2
5        2          1
6        6          9
7        8          8
8        5          3
9        4          4
10       4          4
11       0          7
12       5          9
13       0          0

In [6]:
f1_score(y_test, predictions, average="micro")

0.42857142857142855